In [ ]:
!cp /content/drive/MyDrive/product_detection_cleaned.zip /content/
!unzip /content/product_detection_cleaned.zip  -d /content/detection

In [ ]:
!pip3 install ultralytics
!pip3 install transformers datasets -q

In [1]:

# from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import random
import torch
# from PIL import Image

# detection_model = YOLO('/content/drive/MyDrive/yolov8-1280-1-cleaned-dataset.pt')

embeddings_database = []
image_database = []


In [2]:
from transformers import AutoFeatureExtractor, AutoModel

model_ckpt = "D:\\work\\vit_training\\vit-base-patch16-224-in21k-finetuned-eurosat\\checkpoint-2577"
model_ckpt = "D:\\work\\vit_training\\vit-base-patch16-224-in21k-finetuned-eurosat\\checkpoint-12886-finetuned-eurosat\\checkpoint-12885"
extractor = AutoFeatureExtractor.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
hidden_dim = model.config.hidden_size

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


d:\Anaconda3\envs\cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


d:\Anaconda3\envs\cuda\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at D:\work\vit_training\vit-base-patch16-224-in21k-finetuned-eurosat\checkpoint-12886-finetuned-eurosat\checkpoint-12885 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torchvision.transforms as T
from PIL import Image

transformation_chain = T.Compose(
    [
        # We first resize the input image to 256x256 and then we take center crop.

        T.Resize((extractor.size['height'], extractor.size['width'])),
        T.CenterCrop(extractor.size["height"]),
        T.Grayscale(num_output_channels=3),
        T.ToTensor(),
        T.Lambda(lambda x: x[:3, :, :]),
        T.Normalize(mean=extractor.image_mean[:3], std=extractor.image_std[:3]),
    ]
)

In [23]:
def compute_scores(emb_one, emb_two):
    """Computes cosine similarity between two vectors."""
    scores = torch.nn.functional.cosine_similarity(emb_one, emb_two)
    # print(scores)
    return scores.numpy().tolist()[0]

def create_embeddings(image):
    """Fetches the `top_k` similar images with `image` as the query."""
    # Prepare the input query image for embedding computation.
    image_transformed = transformation_chain(image).unsqueeze(0)
    new_batch = {"pixel_values": image_transformed.to(device)}

    # Comute the embedding.
    with torch.no_grad():
        query_embeddings = model(**new_batch).last_hidden_state[:, 0].cpu()
    return query_embeddings


def compare_two_embeddings(embedding1 , embedding2):
  return compute_scores(embedding1, embedding2)


def fetch_similar( all_embeddings , query_embeddings):

    index_of_similar_images =[]
    for i,initial_embedding in enumerate(all_embeddings):
      print(type(initial_embedding) , type(query_embeddings))
      sim_scores = compute_scores(initial_embedding[1], query_embeddings)
      print(sim_scores)
      if sim_scores > 0.7:
        # index_of_similar_images.append(sim_scores)
        index_of_similar_images.append(i)
    return index_of_similar_images


In [5]:
def register_product(name="test" , image_name=""):
  # Open the image file
  image = cv2.imread(image_name)
  image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  # image = Image.open(image_name)
  embeddings = create_embeddings(image)

  embeddings_database.append((name , embeddings))
  image_database.append((name , image))




In [16]:
embeddings_database = []
image_database = []

In [18]:
# register_product(name="wahab1" , image_name='D:\\work\\vit_training\\val\\wahab\\1.jpg')
# register_product(name="wahab2" , image_name='D:\\work\\vit_training\\val\\wahab\\2.jpg')
# register_product(name="wahab3" , image_name='D:\\work\\vit_training\\val\\wahab\\3.jpg')
# register_product(name="wahab4" , image_name='D:\\work\\vit_training\\val\\wahab\\4.jpg')
register_product(name="obama1" , image_name='D:\\work\\vit_training\\val\\obama\\1.jpg')
register_product(name="obama2" , image_name='D:\\work\\vit_training\\val\\obama\\2.jpg')
register_product(name="jlo" , image_name='D:\\work\\vit_training\\val\\jlo\\1.jpg')


In [8]:
embeddings_database

[('wahab1',
  tensor([[-5.3508e-01, -7.6444e-01,  7.6504e-01, -2.2060e-01, -4.0612e+00,
           -3.5778e-01,  1.8357e+00, -3.9068e-01, -1.6934e+00,  1.5800e+00,
           -1.3937e+00,  7.2951e-01,  1.6648e+00, -1.4233e-01, -2.0013e-01,
            5.2097e-01, -2.1355e+00,  1.1194e+00, -3.9468e+00,  1.1689e+00,
           -7.2037e-01,  2.2672e-01,  2.6515e+00, -2.3652e+00,  1.3442e+00,
           -1.0857e+00, -1.0079e+00,  2.3858e+00, -1.2520e+00,  7.4058e-01,
            2.3477e+00, -3.4962e-01, -1.8119e+00, -9.8110e-01, -3.8280e-01,
            1.0284e+00, -6.0064e-01,  3.8099e-01,  5.4853e-01,  1.9594e+00,
           -9.7842e-01,  4.5964e-01,  1.2848e+00, -1.0071e+00,  9.8114e-01,
            4.4790e-01, -2.1282e-01,  6.1857e-01, -2.9256e+00,  1.1035e-01,
            9.2805e-01, -2.8782e+00,  2.1411e+00,  7.6092e-01,  1.3569e+00,
            1.4970e-01,  2.1890e+00,  1.3845e+00,  1.1995e+00, -1.2651e+00,
           -7.9790e-01, -6.9998e-01, -9.6298e-01, -1.4334e+00, -1.6457e+00,
